In [3]:
! wget https://nlp.cs.washington.edu/ambigqa/data/ambignq_light.zip
! unzip ambignq_light.zip

--2021-01-08 17:21:10--  https://nlp.cs.washington.edu/ambigqa/data/ambignq_light.zip
Resolving nlp.cs.washington.edu (nlp.cs.washington.edu)... 128.208.3.120, 2607:4000:200:12::78
Connecting to nlp.cs.washington.edu (nlp.cs.washington.edu)|128.208.3.120|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1061383 (1.0M) [application/zip]
Saving to: ‘ambignq_light.zip’

ambignq_light.zip   100%[===================>]   1.01M  2.62MB/s    in 0.4s    

2021-01-08 17:21:10 (2.62 MB/s) - ‘ambignq_light.zip’ saved [1061383/1061383]

Archive:  ambignq_light.zip
  inflating: LICENSE                 
  inflating: train_light.json        
  inflating: dev_light.json          


In [42]:
import json

In [43]:
file = open('train_light.json')
data_ = json.load(file)

In [44]:
print(data_[0])

{'annotations': [{'type': 'multipleQAs', 'qaPairs': [{'question': 'When did the Simpsons first air on television as an animated short on the Tracey Ullman Show?', 'answer': ['April 19, 1987']}, {'question': 'When did the Simpsons first air as a half-hour prime time show?', 'answer': ['December 17, 1989']}]}], 'id': '-4469503464110108318', 'question': 'When did the simpsons first air on television?'}


In [45]:
que, ans = [], []
for item in data_:
    for anot in item['annotations']:
        if anot['type'] == 'multipleQAs':
            for pair in anot['qaPairs']:
                for answ in pair['answer']:
                    que.append(pair['question'])
                    ans.append(answ)
        else:
            for answ in anot['answer']:
                que.append(item['question'])
                ans.append(answ)

In [46]:
import pandas as pd

In [47]:
df = pd.DataFrame(list(zip(que, ans)), columns =['questions', 'answers']) 

In [48]:
print(df.shape)
df = df.head(10000)

(27822, 2)


In [49]:
! python3 -m spacy download en
# ! python3 -m spacy download de

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [50]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F


from torchtext.data import Field, BucketIterator
from torchtext import data

import spacy
import numpy as np

import random
import math
import time
from tqdm import tqdm

In [51]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [52]:
spacy_en = spacy.load('en')

In [53]:
def tokenize_en(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]

In [54]:
SRC = Field(sequential = True, tokenize = tokenize_en, init_token = '<sos>', eos_token = '<eos>', lower = True)
TRG = Field(sequential = True, tokenize = tokenize_en, init_token = '<sos>', eos_token = '<eos>', lower = True)

In [55]:
fields = [('que', SRC), ('ans', TRG)]

In [56]:
example = [data.Example.fromlist([df.questions[i],df.answers[i]], fields) for i in range(df.shape[0])] 

In [57]:
AmbiQDataset = data.Dataset(example, fields)

In [58]:
print(vars(AmbiQDataset.examples[0]))

{'que': ['when', 'did', 'the', 'simpsons', 'first', 'air', 'on', 'television', 'as', 'an', 'animated', 'short', 'on', 'the', 'tracey', 'ullman', 'show', '?'], 'ans': ['april', '19', ',', '1987']}


In [59]:
train_data, valid_data, test_data = AmbiQDataset.split(split_ratio=[0.7, 0.2, 0.1], random_state=random.seed(SEED))

In [60]:
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)

In [61]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [62]:
BATCH_SIZE = 128
train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data),
    sort=False,
    batch_size = BATCH_SIZE,
    device = device
)

In [63]:
# batch = next(iter(train_iterator))

In [64]:
# x = batch.trg[0]
# x

In [65]:
# TRG.vocab.stoi['good']

In [66]:
# x.unsqueeze(1)

In [67]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        batch_size = src.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        encoder_outputs, hidden = self.encoder(src)
                
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            output, hidden = self.decoder(input, hidden, encoder_outputs)
            
            outputs[t] = output
            
            teacher_force = random.random() < teacher_forcing_ratio
            
            top1 = output.argmax(1) 
            
            input = trg[t] if teacher_force else top1

        return outputs

In [68]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout):
        super().__init__()

        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional = True)
        self.fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src):
        
        embedded = self.dropout(self.embedding(src))        
        outputs, hidden = self.rnn(embedded)      
        hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)))
        
        return outputs, hidden

In [69]:
class Attention(nn.Module):
    def __init__(self, enc_hid_dim, dec_hid_dim):
        super().__init__()
        
        self.attn = nn.Linear((enc_hid_dim * 2) + dec_hid_dim, dec_hid_dim)
        self.v = nn.Linear(dec_hid_dim, 1, bias = False)
        
    def forward(self, hidden, encoder_outputs):
        
        batch_size = encoder_outputs.shape[1]
        src_len = encoder_outputs.shape[0]
        
        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)
        
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim = 2))) 
        attention = self.v(energy).squeeze(2)
                
        return F.softmax(attention, dim=1)

In [70]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout, attention):
        super().__init__()

        self.output_dim = output_dim
        self.attention = attention
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.GRU((enc_hid_dim * 2) + emb_dim, dec_hid_dim)
        
        self.fc_out = nn.Linear((enc_hid_dim * 2) + dec_hid_dim + emb_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, encoder_outputs):
        
        input = input.unsqueeze(0)
                
        embedded = self.dropout(self.embedding(input))
                
        a = self.attention(hidden, encoder_outputs)          
        a = a.unsqueeze(1)
                
        encoder_outputs = encoder_outputs.permute(1, 0, 2)        
        weighted = torch.bmm(a, encoder_outputs)        
        weighted = weighted.permute(1, 0, 2)
                
        rnn_input = torch.cat((embedded, weighted), dim = 2)
                    
        output, hidden = self.rnn(rnn_input, hidden.unsqueeze(0))
        assert (output == hidden).all()
        
        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted = weighted.squeeze(0)
        
        prediction = self.fc_out(torch.cat((output, weighted, embedded), dim = 1))
                
        return prediction, hidden.squeeze(0)

In [71]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(SRC.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
ENC_HID_DIM = 512
DEC_HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

attn = Attention(ENC_HID_DIM, DEC_HID_DIM)
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn)

model = Seq2Seq(enc, dec, device).to(device)

In [72]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(4427, 256)
    (rnn): GRU(256, 512, bidirectional=True)
    (fc): Linear(in_features=1024, out_features=512, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (attention): Attention(
      (attn): Linear(in_features=1536, out_features=512, bias=True)
      (v): Linear(in_features=512, out_features=1, bias=False)
    )
    (embedding): Embedding(4427, 256)
    (rnn): GRU(1280, 512)
    (fc_out): Linear(in_features=1792, out_features=4427, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [73]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 16,637,515 trainable parameters


In [74]:
optimizer = optim.Adam(model.parameters())

In [75]:
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [76]:
def train(model, train_iterator, optimizer, criterion, clip):
    model.train()
    epoch_loss = 0
    for i, batch in enumerate(tqdm(train_iterator)):
        src, trg = batch.que, batch.ans
        optimizer.zero_grad()
        output = model(src, trg)
        output_dim = output.shape[-1]
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        loss = criterion(output, trg)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        epoch_loss += loss.item()
    return epoch_loss / len(train_iterator)

In [77]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(tqdm(iterator)):

            src = batch.que
            trg = batch.ans

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)
            
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [78]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [79]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'\nEpoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

  0%|          | 0/55 [00:00<?, ?it/s]


Epoch: 01 | Time: 0m 12s
	Train Loss: 5.192 | Train PPL: 179.812
	 Val. Loss: 3.944 |  Val. PPL:  51.639


  2%|▏         | 1/55 [00:00<00:06,  8.35it/s]


Epoch: 02 | Time: 0m 12s
	Train Loss: 4.372 | Train PPL:  79.170
	 Val. Loss: 3.611 |  Val. PPL:  37.015


  2%|▏         | 1/55 [00:00<00:09,  5.61it/s]

...


Epoch: 08 | Time: 0m 12s
	Train Loss: 2.180 | Train PPL:   8.850
	 Val. Loss: 3.492 |  Val. PPL:  32.839


  2%|▏         | 1/55 [00:00<00:07,  7.48it/s]


Epoch: 09 | Time: 0m 12s
	Train Loss: 1.909 | Train PPL:   6.746
	 Val. Loss: 3.556 |  Val. PPL:  35.021


100%|██████████| 8/8 [00:00<00:00, 10.09it/s]


Epoch: 10 | Time: 0m 12s
	Train Loss: 1.712 | Train PPL:   5.542
	 Val. Loss: 3.606 |  Val. PPL:  36.810
